In [ ]:
case_ID = 0

from pathlib import Path
import gmsh
from mpi4py import MPI
from dolfinx.io import gmshio
import numpy as np

gmsh.initialize()

gmsh.model.add("3Dgeometry")

L1, L2, L3, L4 = 4., 4., 4., 4.
h = 4
angle = 0

meshsize = 0.25

p1 = gmsh.model.geo.addPoint(-L1/2, -L4/2, 0, meshSize=meshsize)
p2 = gmsh.model.geo.addPoint(L1/2, -L2/2, 0, meshSize=meshsize)
p3 = gmsh.model.geo.addPoint(L3/2, L2/2, 0, meshSize=meshsize)
p4 = gmsh.model.geo.addPoint(-L3/2, L4/2, 0, meshSize=meshsize)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

curve_loop_bottom = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])
bottom_surface = gmsh.model.geo.addPlaneSurface([curve_loop_bottom])

gmsh.model.geo.synchronize()
bottom_tag_num = gmsh.model.addPhysicalGroup(2, [bottom_surface], name="bottom_surface")

ov = gmsh.model.geo.twist([(2, bottom_surface)], 0., 0., 0, 0, 0, h, 0, 0, 1,
                          (angle * np.pi / 360.), [10], [], False)

gmsh.model.geo.synchronize()

top_tag_num = gmsh.model.addPhysicalGroup(2, [ov[0][1]], name="top_surface")
volume = gmsh.model.addPhysicalGroup(3, [ov[1][1]], name="Volume")
lateral1_tag_num = gmsh.model.addPhysicalGroup(2, [ov[2][1]], name="lateral_surface1")
lateral2_tag_num = gmsh.model.addPhysicalGroup(2, [ov[3][1]], name="lateral_surface2")
lateral3_tag_num = gmsh.model.addPhysicalGroup(2, [ov[4][1]], name="lateral_surface3")
lateral4_tag_num = gmsh.model.addPhysicalGroup(2, [ov[5][1]], name="lateral_surface4")

gmsh.model.mesh.generate(3)
gmsh.write("twisted_cuboid.vtk")

domain, cell_tags, facet_tags = gmshio.model_to_mesh(gmsh.model, MPI.COMM_WORLD, 0, gdim=3)

gmsh.finalize()

points = domain.geometry.x
cells = domain.topology.connectivity(2, 0).array.reshape((-1, 3))


In [ ]:
t = 0  # Start time
T = 5  # End time
num_steps = 50  # Number of time steps
dt = (T - t) / num_steps  # Time step size

In [ ]:
class exact_solution():
    def __init__(self, t):
        self.t = t

    def __call__(self, x):
        return 1 + x[0]**2 * x[1] + 2 * x[0] * x[1]**2 + \
            x[0] * x[1] * x[2] ** 3 + 2 * self.t
    
u_exact = exact_solution(t=t)

In [ ]:
from dolfinx import fem, default_scalar_type
import ufl

V2 = fem.functionspace(domain, ("Lagrange", 2))

u_ex2 = fem.Function(V2)
u_ex2.interpolate(u_exact)

tdim = domain.topology.dim
fdim = tdim - 1
domain.topology.create_connectivity(fdim, tdim)

dofs_bottom_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(bottom_tag_num))
dofs_lateral1_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(lateral1_tag_num))
dofs_lateral2_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(lateral2_tag_num))
dofs_lateral3_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(lateral3_tag_num))
dofs_lateral4_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(lateral4_tag_num))
dofs_top_b = fem.locate_dofs_topological(V2, fdim, facet_tags.find(top_tag_num))

dofs_drchlt_BC = np.concatenate([dofs_bottom_b])
dofs_neumnn_BC = np.concatenate(
    [dofs_lateral1_b, dofs_lateral2_b, dofs_lateral3_b, dofs_lateral4_b, dofs_top_b])

f_bottom = fem.Function(V2)
# f_left.interpolate(lambda x: np.full(x.shape[1], 0))
f_bottom.interpolate(u_exact)

drchlt_bc_bottom = fem.dirichletbc(f_bottom, dofs_bottom_b)

In [ ]:
def initial_condition(x):
    return 1 + x[0]**2 * x[1] + 2 * x[0] * x[1]**2 + x[0] * x[1] * x[2] ** 3

u_n = fem.Function(V2)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

In [ ]:
x = ufl.SpatialCoordinate(domain)
src_term = 2 - (4 * x[0] + 2 * x[1] + 6 * x[0] * x[1] * x[2])

g_lateral1 = (x[0] ** 2 + 4 * x[0] * x[1] + x[0] * x[2] ** 3)
g_lateral2 = -(2 * x[0] * x[1] + 2 * x[1] ** 2 + x[1] * x[2] ** 3)
g_lateral3 = -(x[0] ** 2 + 4 * x[0] * x[1] + x[0] * x[2] ** 3)
g_lateral4 = (2 * x[0] * x[1] + 2 * x[1] ** 2 + x[1] * x[2] ** 3)
g_top = -(3 * x[0] * x[1] * x[2] ** 2)

ds = ufl.Measure("ds", domain=domain, subdomain_data=facet_tags)

In [ ]:
u, v = ufl.TrialFunction(V2), ufl.TestFunction(V2)
a = dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx \
    + u * v * ufl.dx
L = (u_n + dt * src_term) * v * ufl.dx \
        + dt * (- g_lateral1 * v * ds(lateral1_tag_num) \
                - g_lateral2 * v * ds(lateral2_tag_num) \
                - g_lateral3 * v * ds(lateral3_tag_num) \
                - g_lateral4 * v * ds(lateral4_tag_num) \
                - g_top * v * ds(top_tag_num))

bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [ ]:
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, \
    create_vector, apply_lifting, set_bc

from petsc4py import PETSc

A = assemble_matrix(bilinear_form, bcs=[drchlt_bc_bottom])
A.assemble()
b = create_vector(linear_form)
uh2 = fem.Function(V2)

solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.GMRES)
solver.getPC().setType(PETSc.PC.Type.ILU) 
solver.setTolerances(rtol=1e-12)

In [ ]:
V1 = fem.functionspace(domain, ("Lagrange", 1))
u_ex1 = fem.Function(V1)
uh1 = fem.Function(V1)
uh1.interpolate(u_exact)
uh1s = np.zeros((points.shape[0], num_steps + 1))
uh1s[:, 0] = uh1.x.petsc_vec.getArray()


from dolfinx import io
results_folder = Path("./results/u") / f"case_ID_{case_ID}"
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / f"case_ID_{case_ID}"
xdmf = io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w")
xdmf.write_mesh(domain)
xdmf.write_function(uh1, u_exact.t)

for n in range(num_steps):
    # Update Diriclet boundary condition
    u_exact.t += dt
    f_bottom.interpolate(u_exact)
    u_ex2.interpolate(u_exact)

    # A.zeroEntries()
    # assemble_matrix(A, bilinear_form, bcs=[drchlt_bc_left, drchlt_bc_right])
    # A.assemble()
    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)

    # Apply Dirichlet boundary condition to the vector
    apply_lifting(b, [bilinear_form], [[drchlt_bc_bottom]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [drchlt_bc_bottom])

    # Solve linear problem
    solver.solve(b, uh2.x.petsc_vec)
    print(f"n: {n}, Final residual norm: {solver.getResidualNorm()}")
    uh2.x.scatter_forward()

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = uh2.x.array

    u_ex1.interpolate(u_exact)
    uh1.interpolate(uh2)
    uh1s[:, n + 1] = uh1.x.petsc_vec.getArray()
    error_L2 = np.sqrt(domain.comm.allreduce(fem.assemble_scalar(fem.form((uh2 - u_ex2)**2 * ufl.dx)), op=MPI.SUM))
    error_max = domain.comm.allreduce(np.max(np.abs(uh2.x.array - u_ex2.x.array)), op=MPI.MAX)
    if domain.comm.rank == 0:
        print(f"L2-error: {error_L2:.2e}, Error_max: {error_max:.2e}")
        print("--------------------")
        print(" ")
    
    xdmf.write_function(uh1, u_exact.t)

xdmf.close()

In [ ]:
udiff = fem.Function(V1)
udiff.x.petsc_vec.setArray(np.abs(uh1.x.petsc_vec.getArray() - u_ex1.x.petsc_vec.getArray()))# / (uh.vector.getArray()).max())

results_folder = Path("./results/error") / f"case_ID_{case_ID}"
results_folder.mkdir(exist_ok=True, parents=True)
filename = results_folder / f"case_ID_{case_ID}"

with io.XDMFFile(domain.comm, filename.with_suffix(".xdmf"), "w") as xdmf:
    xdmf.write_mesh(domain)
    xdmf.write_function(udiff)

In [ ]:
drchlt_bool = np.zeros(points.shape[0])
neumnn_bool = np.zeros(points.shape[0])

dofs_bottom_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(bottom_tag_num))
dofs_lateral1_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(lateral1_tag_num))
dofs_lateral2_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(lateral2_tag_num))
dofs_lateral3_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(lateral3_tag_num))
dofs_lateral4_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(lateral4_tag_num))
dofs_top_b = fem.locate_dofs_topological(V1, fdim, facet_tags.find(top_tag_num))

dofs_drchlt_BC = np.concatenate([dofs_bottom_b])
dofs_neumnn_BC = np.concatenate(
    [dofs_lateral1_b, dofs_lateral2_b, dofs_lateral3_b, dofs_lateral4_b, dofs_top_b])
drchlt_bool[dofs_drchlt_BC] = 1.
neumnn_bool[dofs_neumnn_BC] = 1.

data = np.concatenate(
    [points[:, 0].reshape((-1, 1)),
     points[:, 1].reshape((-1, 1)),
     drchlt_bool.reshape((-1, 1)),
     neumnn_bool.reshape((-1, 1)),
     uh1s], axis=1)

data_folder = Path("./data")
data_folder.mkdir(exist_ok=True, parents=True)
filename = data_folder / f"case_ID_{case_ID}"

np.save(filename, data)